# Notebook 2 — Chatbot RAG (con retrieval y citas)

Pipeline: **PyPDFLoader → RecursiveCharacterTextSplitter → OpenAIEmbeddings → Chroma → RetrievalQA**

**Objetivo:**

- Construir un Pipeline de Indexación RAG: Demostrar cómo procesar documentos PDF (PyPDFLoader), dividirlos en fragmentos (RecursiveCharacterTextSplitter) y convertirlos en vectores (HuggingFaceEmbeddings) almacenados en una base de datos vectorial (Chroma).

- Implementar Búsqueda Semántica: Enseñar a construir un retriever que encuentre los fragmentos más relevantes basándose en el significado (vectores), como solución a la fragilidad de la búsqueda léxica (difflib) del notebook anterior.

- Demostrar "Grounding" y Citas: Construir una cadena RAG (LCEL) que fuerce al LLM a basar sus respuestas únicamente en los fragmentos de texto recuperados ({context}) y muestre las fuentes (citas) de donde provino la información.

- Comparar Estrategias de Búsqueda: Introducir y comparar dos métodos de retrieval: la búsqueda por similitud estándar (similarity) y la búsqueda por Relevancia Marginal Máxima (mmr) para obtener resultados más diversos.

- Crear una UI Interactiva Completa: Usar gradio (gr.Blocks) para construir una aplicación web que permita al usuario cargar sus propios PDFs, procesarlos y chatear con ellos, incluyendo un selector para comparar los dos tipos de RAG.

> Requisitos: una API key de OpenAI (`OPENAI_API_KEY`) y 1–3 PDFs cortos del curso para pruebas.

## 0) Instalación y preparación del entorno
Instalaremos los paquetes necesarios y configuraremos la clave de API.

In [1]:
# ⬇️ 1. Instalar dependencias
# (Instala/actualiza todas las bibliotecas necesarias en un solo comando)
!pip install -qU \
    langchain \
    langchain-openai \
    langchain-community \
    langchain-text-splitters \
    chromadb \
    tiktoken \
    pypdf \
    gradio \
    sentence-transformers

# ⬇️ 2. Importar y verificar
import langchain
from langchain_core.runnables import RunnablePassthrough

# Imprime la versión instalada para confirmar que se cargó la más reciente
print(f"LangChain version instalada: {langchain.__version__}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 119.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.9/469.9 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 12.1 MB/s eta 0:

In [12]:
import os
from google.colab import userdata

# --- Instrucciones SECRETS DE GOOGLE ---
# 1. Haz clic en el icono de la llave (🔑) en el panel izquierdo.
# 2. Haz clic en 'Agregar un nuevo secreto'.
# 3. Nombre (Name): OPENAI_API_KEY
# 4. Valor (Value): psk... (pega tu clave aquí)
# 5. Activa el interruptor (toggle) para 'Acceso al notebook'.
# 6. ¡Listo! Ahora puedes ejecutar esta celda.

# Obtiene la clave de forma segura desde los secretos de Colab
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Asigna la clave al entorno de ejecución
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# Verifica que se haya cargado (imprime la longitud, no la clave)
if OPENAI_API_KEY:
    print('API key cargada desde Secretos (longitud):', len(OPENAI_API_KEY))
else:
    print('ERROR: No se encontró el secreto "OPENAI_API_KEY".')

API key cargada desde Secretos (longitud): 164


## 1) Cargar documentos PDF (DocumentLoader)
Sube 1–3 PDFs (guías, programas, papers). Se extraerá su texto y metadatos (archivo y número de página).

In [20]:
from google.colab import files

# 1. Da una instrucción clara al usuario ANTES de mostrar el botón
print("Por favor, sube uno o más archivos PDF para procesar...")

# 2. Abre la ventana de diálogo para cargar archivos
uploaded = files.upload()

# 3. Obtiene la lista de nombres de archivo
pdf_paths = list(uploaded.keys())

# 4. Verifica si se subieron archivos y da retroalimentación
if pdf_paths:
    print(f"\n¡Éxito! Se cargaron {len(pdf_paths)} archivo(s):")
    # Imprime los nombres de los archivos cargados
    for i, path in enumerate(pdf_paths, 1):
        print(f"  {i}. {path}")
        print(f"{pdf_paths}")
else:
    print("\nCarga cancelada. No se seleccionó ningún archivo.")



Por favor, sube uno o más archivos PDF para procesar...


Saving Lee et al. - 2025 - Research Trends and Challenges in Diagnostic Classification Models Insights from Dynamic Topic Mode.pdf to Lee et al. - 2025 - Research Trends and Challenges in Diagnostic Classification Models Insights from Dynamic Topic Mode.pdf
Saving Lee et al. - 2025 - AI-powered Automatic Item Generation for Psychological Tests A Conceptual Framework for an LLM-base.pdf to Lee et al. - 2025 - AI-powered Automatic Item Generation for Psychological Tests A Conceptual Framework for an LLM-base (1).pdf

¡Éxito! Se cargaron 2 archivo(s):
  1. Lee et al. - 2025 - Research Trends and Challenges in Diagnostic Classification Models Insights from Dynamic Topic Mode.pdf
['Lee et al. - 2025 - Research Trends and Challenges in Diagnostic Classification Models Insights from Dynamic Topic Mode.pdf', 'Lee et al. - 2025 - AI-powered Automatic Item Generation for Psychological Tests A Conceptual Framework for an LLM-base (1).pdf']
  2. Lee et al. - 2025 - AI-powered Automatic Item Genera

In [21]:
# Importa la clase específica de LangChain para cargar (leer) archivos PDF.
from langchain_community.document_loaders import PyPDFLoader

# Inicializa una lista vacía. 'docs' guardará CADA PÁGINA de TODOS los PDFs
# como un objeto 'Document' separado.
docs = []

# Inicia un bucle que recorre la lista 'pdf_paths' (que contiene los nombres
# de los archivos que subiste, ej: ['doc1.pdf', 'doc2.pdf']).
for path in pdf_paths:

    # 1. Crea una instancia del cargador, apuntando al archivo PDF actual ('path').
    loader = PyPDFLoader(path)

    # 2. Ejecuta la carga. .load() lee el PDF y devuelve una lista ('d')
    #    donde cada elemento es una página del PDF (objeto Document).
    d = loader.load()  # cada página → Document(text, metadata)

    # 3. Añade las páginas de la lista 'd' a la lista principal 'docs'.
    #    (Usamos .extend() para añadir los elementos de la lista, no la lista en sí).
    docs.extend(d)

# Imprime un mensaje de confirmación mostrando el número total de páginas cargadas.
print(f'Total de páginas cargadas: {len(docs)}')

# Muestra los metadatos de la primera página (docs[0]) como inspección.
# Los metadatos suelen incluir {'source': 'nombre_del_archivo.pdf', 'page': 0}.
# 'if docs else {}' evita un error si la lista 'docs' está vacía.
docs[0].metadata if docs else {}

Total de páginas cargadas: 63


{'producer': 'PDFlib+PDI 9.2.0 (C++/Win64)',
 'creator': 'PTC Arbortext Publishing Engine',
 'creationdate': '2025-05-07T16:18:41+05:30',
 'keywords': 'Diagnosis classification models (DCMs); cognitive diagnosis models (CDMs); dynamic topic modeling (DTM); review; research trends',
 'moddate': '2025-05-07T16:18:41+05:30',
 'title': 'Research Trends and Challenges in Diagnostic Classification Models: Insights from Dynamic Topic Mode',
 'rgid': 'PB:391405685_AS:11431281484469514@1749095249386',
 'source': 'Lee et al. - 2025 - Research Trends and Challenges in Diagnostic Classification Models Insights from Dynamic Topic Mode.pdf',
 'total_pages': 34,
 'page': 0,
 'page_label': '1'}

## 2) Chunking (TextSplitter)
Dividimos cada página en **fragmentos** para mejorar la recuperación semántica. Puedes ajustar `chunk_size` y `chunk_overlap`.

In [22]:
# Importa la clase recomendada por LangChain para dividir texto.
# 'Recursive' (recursivo) significa que intentará dividir el texto de forma
# inteligente usando una lista de separadores, desde el más grande al más pequeño.
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Crea una instancia del divisor de texto con parámetros específicos.
splitter = RecursiveCharacterTextSplitter(

    # Define el tamaño máximo de cada 'chunk' (fragmento) en caracteres.
    # 700 es un tamaño común, lo suficientemente grande para tener contexto,
    # pero lo suficientemente pequeño para ser preciso en la búsqueda.
    chunk_size=700,

    # Define cuántos caracteres se solaparán entre chunks consecutivos.
    # 120 es crucial para evitar que una idea se corte bruscamente
    # entre el final de un chunk y el inicio del siguiente.
    chunk_overlap=120,

    # Define la lista de separadores que el splitter intentará usar, EN ORDEN.
    # 1ro intentará dividir por doble salto de línea (párrafos).
    # Si un chunk sigue siendo muy grande, intentará por salto de línea simple, etc.
    separators=['\n\n', '\n', '. ', ' ']
)

#

# 2. Ejecuta la división.
# Toma la lista 'docs' (que contiene todas las páginas) y aplica las reglas
# de división para generar la nueva lista de 'chunks'.
chunks = splitter.split_documents(docs)

# Imprime un recuento de cuántos chunks se generaron en total.
# (Es normal que 'len(chunks)' sea mucho mayor que 'len(docs)').
print('Chunks generados:', len(chunks))

# 3. Inspecciona el primer chunk.
# Muestra los metadatos del primer fragmento (chunks[0]).
# Verás que LangChain ha conservado los metadatos originales
# (ej. 'source' y 'page') del documento del que proviene.
chunks[0].metadata if chunks else {}

Chunks generados: 402


{'producer': 'PDFlib+PDI 9.2.0 (C++/Win64)',
 'creator': 'PTC Arbortext Publishing Engine',
 'creationdate': '2025-05-07T16:18:41+05:30',
 'keywords': 'Diagnosis classification models (DCMs); cognitive diagnosis models (CDMs); dynamic topic modeling (DTM); review; research trends',
 'moddate': '2025-05-07T16:18:41+05:30',
 'title': 'Research Trends and Challenges in Diagnostic Classification Models: Insights from Dynamic Topic Mode',
 'rgid': 'PB:391405685_AS:11431281484469514@1749095249386',
 'source': 'Lee et al. - 2025 - Research Trends and Challenges in Diagnostic Classification Models Insights from Dynamic Topic Mode.pdf',
 'total_pages': 34,
 'page': 0,
 'page_label': '1'}

## 3) Embeddings y Vector Store (Chroma)
Convertimos cada fragmento en un **embedding** y lo indexamos en **Chroma** para búsqueda por similitud.
Puedes cambiar el nombre de la colección y activar persistencia en disco si lo deseas.

In [23]:
# Importa la clase para usar embeddings gratuitos de HuggingFace.
from langchain_community.embeddings import HuggingFaceEmbeddings
# Importa la clase Chroma, que es nuestra base de datos vectorial.
from langchain_community.vectorstores import Chroma

# 1. Configura el modelo de embedding gratuito

# Define el nombre del modelo gratuito que se descargará (uno popular y ligero).
model_name = "sentence-transformers/all-MiniLM-L6-v2"
# Crea la instancia del 'embedder' (incrustador), cargando el modelo gratuito.
EMBEDDINGS = HuggingFaceEmbeddings(model_name=model_name)

# Imprime un mensaje de estado para confirmar qué embedder se está usando.
print("Usando embeddings gratuitos de HuggingFace...")

# Define un nombre para la "colección" (la tabla) dentro de ChromaDB.
COLLECTION_NAME = 'rag_curso'
# Define la ruta para guardar la BD (None = solo en memoria, no persistente).
PERSIST_DIR = None

# 2. Indexa los documentos en la Vector Store
#

# Este es el paso clave: crea la base de datos vectorial en memoria.
vectordb = Chroma.from_documents(
    documents=chunks,         # La lista de fragmentos de texto (chunks) a indexar.
    embedding=EMBEDDINGS,     # El modelo de embedding a usar (el de HuggingFace).
    collection_name=COLLECTION_NAME, # El nombre para esta colección de vectores.
    persist_directory=PERSIST_DIR  # Dónde guardar (en este caso, 'None' o memoria).
)

# Imprime un recuento de cuántos chunks se indexaron exitosamente.
print('Vector DB listo. Documentos indexados:', vectordb._collection.count())

Usando embeddings gratuitos de HuggingFace...
Vector DB listo. Documentos indexados: 406


## 4) Crear el Retriever y la cadena RetrievalQA
El *retriever* recupera los **k** fragmentos más relevantes. `RetrievalQA` usa esos fragmentos como contexto para el LLM y además nos devuelve las **fuentes**.

In [28]:
#--- 1. Importaciones de LangChain ---

# Importa el conector para los modelos de Chat de OpenAI (ej. GPT-4o).
from langchain_openai import ChatOpenAI
# Importa la clase para crear plantillas de prompts de chat.
from langchain_core.prompts import ChatPromptTemplate
# Importa componentes de LCEL (LangChain Expression Language).
from langchain_core.runnables import RunnablePassthrough
# Importa una función de Python para obtener claves de un diccionario.
from operator import itemgetter

# --- 2. Configuración del LLM ---

# Crea una instancia del modelo de lenguaje.
llm = ChatOpenAI(
    model='gpt-4o-mini',  # Especifica el modelo de OpenAI a usar.
    temperature=0         # Fija la creatividad en 0 para respuestas consistentes.
)

# --- 3. Configuración del Retriever ---

# Crea un objeto 'retriever' (buscador) a partir de la base de datos vectorial.
# (Asume que 'vectordb' fue creada en una celda anterior).
retriever = vectordb.as_retriever(
    # Define el tipo de búsqueda. 'similarity' es la búsqueda estándar.
    search_type='similarity',
    # Define cuántos documentos ('k') debe recuperar.
    search_kwargs={'k': 4}
)

# --- 4. Definición del Prompt (Plantilla de Instrucciones) ---

# Define la plantilla de texto que se le enviará al LLM.
prompt = ChatPromptTemplate.from_template(

 """Responde las preguntas de manera clara.
 La respuesta debe ser muy bien argumentada, empleando lenguaje técnico de máximo 400 caracteres, agrega al menos una referencia a los documentos en formato APA7 y basada
 en este contexto:

<context>
{context}
</context>

Question: {input}""")

# Esta es la forma moderna y correcta de construir la cadena RAG en LCEL
# para poder acceder tanto a la respuesta como a las fuentes (contexto).
qa = (
    # 1. Define un diccionario paralelo para procesar la entrada.
    {
        # Clave 'context': Toma la entrada ('input'), la pasa (|) al 'retriever' y guarda los documentos resultantes.
        "context": itemgetter("input") | retriever,
        # Clave 'input': Toma la entrada ('input') y la pasa directamente, sin cambios.
        "input": itemgetter("input"),
    }
    # 2. Pasa el diccionario {'context', 'input'} a la siguiente etapa.
    | RunnablePassthrough.assign(
        # 3. Asigna una nueva clave ('answer') al diccionario.
        #    Para obtener su valor, pasa el diccionario al 'prompt' y luego el 'prompt' al 'llm'.
        answer=(prompt | llm)
    )
)

# 4. Resultado final:
#    Al invocar 'qa', la salida será un diccionario completo:
#    {
#      'context': [Documentos recuperados],
#      'input': 'Pregunta del usuario',
#      'answer': <AIMessage con la respuesta del LLM>
#    }

# Imprime la cadena para verificar su estructura
print(f"{qa}")

first={
  context: RunnableLambda(itemgetter('input'))
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7c030f484770>, search_kwargs={'k': 4}),
  input: RunnableLambda(itemgetter('input'))
} middle=[] last=RunnableAssign(mapper={
  answer: ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Responde las preguntas de manera clara.\n La respuesta debe ser muy bien argumentada, empleando lenguaje técnico de máximo 400 caracteres, agrega al menos una referencia a los documentos en formato APA7 y basada\n en este contexto:\n\n<context>\n{context}\n</context>\n\nQuestion: {input}'), additional_kwargs={})])
          | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7c03

## 5) Consulta A. Ejemplo de **grounding** con fuentes

Prueba con una pregunta del PDF, procura que exista información sobre ella. La salida mostrará la respuesta del LLM y **los documentos fuente** (archivo y página).

In [29]:

# Define una función 'ask' que recibe la pregunta del usuario ('query').
def ask(query: str):

    # 1. Ejecuta la cadena 'qa' (definida antes) con la consulta del usuario.
    #    'out' es el diccionario: {'context': ..., 'input': ..., 'answer': ...}
    out = qa.invoke({'input': query})

    # 2. Extrae la respuesta de texto (el .content) del objeto 'answer' (AIMessage).
    ans = out['answer'].content

    # 3. Extrae la lista de documentos (fuentes) de la clave 'context'.
    srcs = out.get('context', [])

    # 4. Imprime la respuesta principal generada por el LLM.
    print('Respuesta similarity:\n', ans)

    # Imprime el encabezado para la sección de citas.
    print('\n🔎 Fuentes:')

    # 5. Inicia un bucle (enumerado desde 1) por cada documento 'd' en las fuentes 'srcs'.
    for i, d in enumerate(srcs, 1):

        # Obtiene los metadatos del documento (o un dict vacío si no hay).
        meta = d.metadata or {}
        # Obtiene el nombre del archivo ('source') de los metadatos.
        src = meta.get('source', 'desconocido')
        # Obtiene el número de página de los metadatos.
        page = meta.get('page', '¿?')

        # Crea un 'snippet' (fragmento) corto del contenido del documento.
        snippet = (d.page_content[:220] + '...') if len(d.page_content) > 220 else d.page_content

        # Imprime la cita formateada (número, archivo, página y snippet).
        print(f'[{i}] {src} — página {page}\n    └─ "{snippet}"\n')

# 6. Llama a la función 'ask' con una pregunta de prueba para ejecutar todo el flujo RAG.
ask('¿Qué es la Generación Automática de ítems?')

Respuesta similarity:
 La Generación Automática de Ítems (AIG, por sus siglas en inglés) es un proceso que utiliza modelos de lenguaje avanzados para crear preguntas o ítems de evaluación de manera eficiente y escalable. Este enfoque se basa en agentes autónomos que generan y revisan ítems, asegurando objetividad y reduciendo sesgos. La AIG es crucial en la psicometría y la evaluación de personal, optimizando la creación de pruebas y mejorando la calidad de los ítems generados (Lee et al., 2025). 

Referencia: Lee, P. (2025). AI-powered Automatic Item Generation for Psychological Tests: A Conceptual Framework for an LLM-based Multi-Agent AIG System. *Journal of Business and Psychology*. https://doi.org/10.1007/s10869-025-10067-y

🔎 Fuentes:
[1] planificación_curso-IA_Avanzado.pdf — página 0
    └─ "guiada
Archivo .py o notebook con 
ejemplos de consultas y 
resultados comentados
29 de 
octubre
4:00 p.m. - 
7:00 p.m.
2. Asistentes 
personalizados
Bot con RAG y 
Chatbot con 
FAQ 
Prácti

## 6) Ajustes de recuperación (k, search_type, filtros)
- **k:** número de fragmentos recuperados.
- **search_type:** `'similarity'` o `'mmr'` (Maximum Marginal Relevance para más diversidad).
- **Filtros por metadatos:** si tus documentos tienen metadatos personalizados, se pueden pasar en `search_kwargs`.

In [30]:
# Importa la clase que necesitamos para mantener el contexto.
from langchain_core.runnables import RunnablePassthrough


# Importa la clase para crear plantillas de prompts.
from langchain_core.prompts import ChatPromptTemplate
# Importa la clase para "pasar" datos (aunque no se usa en ESTA versión de la cadena).
from langchain_core.runnables import RunnablePassthrough
# Importa la función para obtener claves de un diccionario.
from operator import itemgetter

# 1. Crear un NUEVO retriever que usa 'MMR'
# (Asume que 'vectordb' ya existe en memoria).
retriever_mmr = vectordb.as_retriever(
    # 'mmr' (Maximum Marginal Relevance) busca chunks que sean relevantes Y diversos
    # (evitando que los 6 chunks digan lo mismo).
    search_type='mmr',
    search_kwargs={
        'k': 4,             # Le pedimos que recupere 6 chunks.
        'lambda_mult': 0.5  # Un balance (0.5) entre relevancia y diversidad.
        }
)

# 2. Definir un NUEVO prompt, esta vez con reglas muy estrictas.
prompt = ChatPromptTemplate.from_template("""Responde las preguntas de manera clara.
 La respuesta debe ser muy bien argumentada, empleando lenguaje técnico de máximo 400 caracteres, agrega al menos una referencia a los documentos en formato APA7 y basada
 en este contexto:
<context>
{context}
</context>

Question: {input}""")


# Construimos 'qa_mmr' usando el mismo patrón que 'qa' (para que devuelva las fuentes).
qa_mmr = (
    {
        # 1. Obtiene el contexto usando el retriever MMR.
        "context": itemgetter("input") | retriever_mmr,
        # 2. Pasa la pregunta original.
        "input": itemgetter("input"),
    }
    # 3. Mantiene el diccionario {'context', 'input'} Y le añade una nueva clave: 'answer'.
    | RunnablePassthrough.assign(
        # 4. La clave 'answer' se genera pasando el diccionario al prompt y luego al LLM.
        answer=(prompt | llm)
    )
)

# Ahora 'out' será un diccionario: {'context': ..., 'input': ..., 'answer': ...}
out = qa_mmr.invoke({'input': '¿Qué es la Generación Automática de ítems?'})

# Ahora puedes imprimir la respuesta Y las fuentes, igual que en tu función 'ask'
print("Respuesta Relevancia Marginal Máxima (MMR):\n", out['answer'].content)
print("\n🔎 Fuentes (MMR):")
# Itera sobre los documentos de 'context' y los imprime.
for i, d in enumerate(out['context'], 1):
    print(f"  [{i}] {d.metadata.get('source', '?')} (pág. {d.metadata.get('page', '?')})")

Respuesta Relevancia Marginal Máxima (MMR):
 La Generación Automática de Ítems (GAI) es un proceso que utiliza modelos de lenguaje, como los de inteligencia artificial, para crear preguntas y evaluaciones de manera eficiente. Este enfoque permite generar ítems alineados con objetivos específicos, mejorando la calidad y relevancia de las evaluaciones. Sin embargo, es crucial integrar la intervención humana para optimizar la calidad de los ítems generados, evitando estancamientos en el proceso (Lee et al., 2025). 

Referencia: Lee, P. (2025). AI-powered Automatic Item Generation for Psychological Tests: A Conceptual Framework for an LLM-based Multi-Agent AIG System. *Journal of Business and Psychology*. https://doi.org/10.1007/s10869-025-10067-y

🔎 Fuentes (MMR):
  [1] planificación_curso-IA_Avanzado.pdf (pág. 0)
  [2] Lee et al. - 2025 - AI-powered Automatic Item Generation for Psychological Tests A Conceptual Framework for an LLM-base (1).pdf (pág. 21)
  [3] Lee et al. - 2025 - AI-pow

## 7) Interfaz rápida con Gradio
Aquí una interfaz sencilla para probar preguntas y ver las **citas** a las fuentes.

In [32]:
# --- 1. Importar todas las bibliotecas necesarias ---
# (Asegúrate de haberlas instalado con !pip install...)
import gradio as gr
import os
from operator import itemgetter
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


# --- 2. Configuración Global (Modelos) ---

# Carga el modelo de embedding gratuito
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Carga el LLM (asume que la API key de OpenAI está en el entorno)
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# Plantilla de Prompt (usaremos la misma para ambos RAGs para una comparación justa)
prompt = ChatPromptTemplate.from_template("""Responde la pregunta de forma concisa (máximo 3 frases), usa APA7 y una referencia textual basándote solo en este contexto:
<context>
{context}
</context>

Pregunta: {input}""")


# --- 3. Función de Indexación (El "Paso 1") ---
# Esta función AHORA CREA AMBAS CADENAS RAG

def process_files(file_list):
    print("Iniciando procesamiento de archivos...")
    if not file_list:
        return None, "❌ Error: No se subieron archivos."

    all_chunks = []

    # 1. Cargar y Dividir los PDFs
    try:
        for file_obj in file_list:
            print(f"  Cargando: {os.path.basename(file_obj.name)}")
            loader = PyPDFLoader(file_obj.name)
            docs = loader.load()

            splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=120)
            chunks = splitter.split_documents(docs)
            all_chunks.extend(chunks)

        print(f"Total de chunks generados: {len(all_chunks)}")

        # 2. Crear la Base de Datos Vectorial (en memoria)
        print("Creando base de datos vectorial (ChromaDB)...")
        vectordb = Chroma.from_documents(
            documents=all_chunks,
            embedding=embeddings
        )

        # 3. Crear el Pipeline RAG (Similarity)
        retriever_sim = vectordb.as_retriever(search_type='similarity', search_kwargs={'k': 4})
        qa_sim = (
            {"context": itemgetter("input") | retriever_sim, "input": itemgetter("input")}
            | RunnablePassthrough.assign(answer=(prompt | llm))
        )
        print("✅ Pipeline RAG (Similarity) listo.")

        # 4. Crear el Pipeline RAG (MMR)
        retriever_mmr = vectordb.as_retriever(search_type='mmr', search_kwargs={'k': 4, 'lambda_mult': 0.5})
        qa_mmr = (
            {"context": itemgetter("input") | retriever_mmr, "input": itemgetter("input")}
            | RunnablePassthrough.assign(answer=(prompt | llm))
        )
        print("✅ Pipeline RAG (MMR) listo.")

        # 5. Guardar AMBAS cadenas en un diccionario
        all_chains = {
            "RAG (Similarity)": qa_sim,
            "RAG (MMR)": qa_mmr
        }

        # Devuelve el diccionario de cadenas Y un mensaje de éxito
        return all_chains, f"✅ ¡Éxito! {len(all_chunks)} chunks procesados. Cadenas 'Similarity' y 'MMR' creadas."

    except Exception as e:
        print(f"Error procesando archivos: {e}")
        return None, f"❌ Error: {e}"


# --- 4. Función de Chat (El "Paso 2") ---
# Esta función AHORA ACEPTA EL TIPO DE RAG

def rag_chat(user_message, history, rag_chain_state, rag_type):

    # Comprueba si el pipeline RAG ("state") ha sido creado
    if rag_chain_state is None:
        return "Por favor, sube uno o más archivos PDF y presiona 'Procesar Archivos' primero."

    # Usa el 'rag_type' para seleccionar la cadena correcta del diccionario 'rag_chain_state'
    try:
        # Elige la cadena correcta del diccionario guardado en el estado
        chain_to_use = rag_chain_state[rag_type]
        print(f"Ejecutando con: {rag_type}")

        # Invoca la cadena seleccionada
        out = chain_to_use.invoke({'input': user_message})

        # Extrae la respuesta y las fuentes
        ans = out['answer'].content
        srcs = out.get('context', [])

        # Formatea las citas
        cites = []
        for d in srcs:
            meta = d.metadata or {}
            src = os.path.basename(meta.get('source','?')) # Solo el nombre del archivo
            page = meta.get('page','?')
            cites.append(f"{src} (pág. {page})")

        footer = '\n\nFuentes:\n- ' + '\n- '.join(dict.fromkeys(cites)) if cites else ''
        return ans + footer

    except Exception as e:
        print(f"Error en 'rag_chat': {e}")
        return f"Error al generar respuesta: {e}"


# --- 5. Construir la Interfaz de Gradio con "Blocks" ---

with gr.Blocks(title="Chatbot RAG con Carga y Selector") as demo:

    # 'gr.State' es la variable "invisible" que guarda el diccionario de pipelines RAG
    rag_chain_state = gr.State(None)

    gr.Markdown("# Chatbot RAG: Carga de Archivos y Comparador (Similarity vs. MMR)")
    gr.Markdown("Sube tus PDFs, presiona 'Procesar Archivos' y luego chatea con tus documentos.")

    with gr.Row():
        with gr.Column(scale=1):
            # Componente de carga de archivos
            file_uploader = gr.File(
                label="Sube tus PDFs",
                file_count="multiple",
                file_types=[".pdf"]
            )

            # Selector de tipo de RAG
            rag_selector = gr.Radio(
                ["RAG (Similarity)", "RAG (MMR)"],
                label="Elige el tipo de Retriever",
                value="RAG (Similarity)" # Valor por defecto
            )

            # Botón para iniciar la indexación
            process_button = gr.Button("Procesar Archivos", variant="primary")

            # Caja de estado para mensajes de éxito o error
            status_box = gr.Textbox(label="Estado", interactive=False)

        with gr.Column(scale=2):
            # Interfaz de chat estándar
            chat_interface = gr.ChatInterface(
                fn=rag_chat,
                # Pasa AMBAS entradas adicionales a la función de chat
                additional_inputs=[rag_chain_state, rag_selector]
            )

    # --- 6. Conectar los Componentes ---

    # Conecta el botón 'process_button' a la función 'process_files'
    process_button.click(
        fn=process_files,
        inputs=[file_uploader],
        outputs=[rag_chain_state, status_box] # La salida se guarda en el estado y el status
    )

print("Lanzando interfaz de Gradio...")
demo.launch(share=False)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Lanzando interfaz de Gradio...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## 8) Buenas prácticas y notas finales
- **Citación obligatoria**: siempre devolver fuentes (archivo y página) para confianza y auditoría.
- **Evaluación**: verifica que los fragmentos recuperados contienen evidencia suficiente.
- **Tamaño de chunk**: prueba 400–900 tokens; solape 10–20%.
- **k y search_type**: ajusta según el corpus. `mmr` puede mejorar diversidad.
- **Persistencia**: usa `persist_directory` para reutilizar índices.
- **Privacidad**: no subas documentos sensibles a Colab sin autorización.